In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import pytest
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection._validation import _backtesting_forecaster
from skforecast.model_selection._split import TimeSeriesFold

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

c:\Users\jaesc2\Miniconda3\envs\skforecast_py13\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
forecaster = ForecasterRecursive(
        regressor=LinearRegression(), lags=3, binner_kwargs={'n_bins': 10}
)
n_backtest = 12
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
            steps                 = 5,
            initial_train_size    = len(y_train),
            window_size           = None,
            differentiation       = None,
            refit                 = False,
            fixed_train_size      = True,
            gap                   = 0,
            skip_folds            = None,
            allow_incomplete_fold = True,
            return_all_indexes    = False,
        )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster              = forecaster,
                                    y                       = y,
                                    exog                    = exog,
                                    cv                      = cv,
                                    metric                  = 'mean_squared_error',
                                    interval                = None,
                                    interval_method         = 'conformal',
                                    use_in_sample_residuals = True,
                                    use_binned_residuals    = True,
                                    random_state            = 123,
                                    return_predictors=     True,
                                    verbose                 = True,
                                    show_progress           = False
                                )

Information of folds
--------------------
Number of observations used for initial training: 38
Number of observations used for backtesting: 12
    Number of folds: 3
    Number skipped folds: 0 
    Number of steps per fold: 5
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 2 observations.

Fold: 0
    Training:   0 -- 37  (n=38)
    Validation: 38 -- 42  (n=5)
Fold: 1
    Training:   No training in this fold
    Validation: 43 -- 47  (n=5)
Fold: 2
    Training:   No training in this fold
    Validation: 48 -- 49  (n=2)



In [4]:
backtest_predictions

,pred,fold,lag_1,lag_2,lag_3,exog
38,0.590596,0,0.893389,0.426351,0.312261,0.304768
39,0.472575,0,0.590596,0.893389,0.426351,0.398186
40,0.530241,0,0.472575,0.590596,0.893389,0.704959
41,0.461633,0,0.530241,0.472575,0.590596,0.995358
42,0.500351,0,0.461633,0.530241,0.472575,0.355915
43,0.491573,1,0.317285,0.115618,0.623953,0.762548
44,0.433273,1,0.491573,0.317285,0.115618,0.593177
45,0.424208,1,0.433273,0.491573,0.317285,0.691702
46,0.533894,1,0.424208,0.433273,0.491573,0.151127
47,0.516931,1,0.533894,0.424208,0.433273,0.398876


In [6]:
5 // 10

0